In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Duchenne_Muscular_Dystrophy"
cohort = "GSE79263"

# Input paths
in_trait_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy"
in_cohort_dir = "../../input/GEO/Duchenne_Muscular_Dystrophy/GSE79263"

# Output paths
out_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/GSE79263.csv"
out_gene_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE79263.csv"
out_clinical_data_file = "../../output/preprocess/Duchenne_Muscular_Dystrophy/clinical_data/GSE79263.csv"
json_path = "../../output/preprocess/Duchenne_Muscular_Dystrophy/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Analysis of gene expression in hTERT/cdk4 immortalized human myoblasts compared to their primary populations in both undifferentiatied (myoblast) and differentiated (myotube) states"
!Series_summary	"hTERT/cdk4 immortalized myogenic human cell lines represent an important tool for skeletal muscle research, being used as therapeutically-pertinent models of various neuromuscular disorders and in numerous fundamental studies of muscle cell function. However, the cell cycle is linked to other cellular processes such as integrin regulation, the PI3K/Akt pathway, and microtubule stability, raising the question as to whether transgenic modification of the cell cycle results in secondary effects that could undermine the validity of these cell models. Here we subjected healthy and disease lines to intensive transcriptomic analysis, comparing immortalized lines with their parent primary populations in both differentiated and undifferentiated states, and tes

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, the dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Duchenne Muscular Dystrophy), we can see it in row 2 with 'disease state'
trait_row = 2

# For age, we can see it in row 4
age_row = 4

# For gender, there's no information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    if pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.lower()
    
    if "duchenne" in value or "dmd" in value:
        return 1
    elif "healthy" in value:
        return 0
    else:
        return None

def convert_age(value):
    if pd.isna(value):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "unknown" in value:
        return None
    
    # Extract numeric values
    import re
    age_match = re.search(r'(\d+)', value)
    if age_match:
        return int(age_match.group(1))
    else:
        return None

# 3. Save Metadata - Initial Filtering
# Determine if trait data is available
is_trait_available = trait_row is not None

validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Try to find and load the clinical data
        # Check if the sample characteristics data is already available from a previous step
        # This assumes that clinical_data was created in a previous step
        # and represents the sample characteristics dictionary shown in the output
        if 'clinical_data' in locals() or 'clinical_data' in globals():
            # Use existing clinical_data variable
            pass
        else:
            # Try different possible file paths/formats for clinical data
            potential_paths = [
                os.path.join(in_cohort_dir, "clinical_data.csv"),
                os.path.join(in_cohort_dir, "sample_characteristics.csv"),
                os.path.join(in_cohort_dir, "characteristics.csv")
            ]
            
            clinical_data = None
            for path in potential_paths:
                if os.path.exists(path):
                    clinical_data = pd.read_csv(path)
                    print(f"Loaded clinical data from {path}")
                    break
            
            if clinical_data is None:
                # If no file is found, create a DataFrame from the sample characteristics dictionary
                # This is a placeholder based on the structure shown in the previous output
                sample_chars = {
                    0: ['cell type: non-myogenic CD56-negative', 'differentiation state: Myoblast', 'differentiation state: Myotube'], 
                    1: ['differentiation state: NA', 'clonal state: Clone', 'clonal state: Primary'], 
                    2: ['clonal state: NA', 'disease state: healthy', 'disease state: Duchenne muscular dystrophy', 'disease state: Healthy'], 
                    3: ['disease state: NA', None], 
                    4: ['age: 80y', 'age: 78y', 'age: unknown', 'age: 79y', 'age: 19y', 'age: 17y', 'age: 15y', 'age: 73y', None]
                }
                
                # Convert the dictionary to a DataFrame
                # This is an approximation - in reality we'd need to know how samples map to these characteristics
                clinical_data = pd.DataFrame(sample_chars)
                print("Created clinical data DataFrame from sample characteristics dictionary")
        
        if clinical_data is not None:
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=None
            )
            
            # Preview the data
            preview = preview_df(selected_clinical_df)
            print("Clinical Data Preview:", preview)
            
            # Create the directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the clinical data
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("Warning: No clinical data could be loaded or created")
    
    except Exception as e:
        print(f"Error in clinical data extraction: {e}")
        print("Continuing with other preprocessing steps...")


Clinical Data Preview: {'GSM2090086': [nan, 80.0], 'GSM2090087': [nan, 78.0], 'GSM2090088': [nan, nan], 'GSM2090089': [nan, 79.0], 'GSM2090090': [nan, 19.0], 'GSM2090091': [nan, 17.0], 'GSM2090092': [nan, 15.0], 'GSM2090093': [nan, 73.0], 'GSM2090094': [0.0, nan], 'GSM2090095': [0.0, nan], 'GSM2090096': [0.0, nan], 'GSM2090097': [0.0, nan], 'GSM2090098': [0.0, nan], 'GSM2090099': [0.0, nan], 'GSM2090100': [0.0, nan], 'GSM2090101': [0.0, nan], 'GSM2090102': [0.0, nan], 'GSM2090103': [0.0, nan], 'GSM2090104': [0.0, nan], 'GSM2090105': [0.0, nan], 'GSM2090106': [0.0, nan], 'GSM2090107': [0.0, nan], 'GSM2090108': [0.0, nan], 'GSM2090109': [0.0, nan], 'GSM2090110': [1.0, nan], 'GSM2090111': [1.0, nan], 'GSM2090112': [1.0, nan], 'GSM2090113': [1.0, nan], 'GSM2090114': [1.0, nan], 'GSM2090115': [1.0, nan], 'GSM2090116': [1.0, nan], 'GSM2090117': [1.0, nan], 'GSM2090118': [1.0, nan], 'GSM2090119': [1.0, nan], 'GSM2090120': [1.0, nan], 'GSM2090121': [1.0, nan], 'GSM2090122': [1.0, nan], 'GSM209

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 62
Header line: "ID_REF"	"GSM2090086"	"GSM2090087"	"GSM2090088"	"GSM2090089"	"GSM2090090"	"GSM2090091"	"GSM2090092"	"GSM2090093"	"GSM2090094"	"GSM2090095"	"GSM2090096"	"GSM2090097"	"GSM2090098"	"GSM2090099"	"GSM2090100"	"GSM2090101"	"GSM2090102"	"GSM2090103"	"GSM2090104"	"GSM2090105"	"GSM2090106"	"GSM2090107"	"GSM2090108"	"GSM2090109"	"GSM2090110"	"GSM2090111"	"GSM2090112"	"GSM2090113"	"GSM2090114"	"GSM2090115"	"GSM2090116"	"GSM2090117"	"GSM2090118"	"GSM2090119"	"GSM2090120"	"GSM2090121"	"GSM2090122"	"GSM2090123"	"GSM2090124"	"GSM2090125"	"GSM2090126"	"GSM2090127"	"GSM2090128"	"GSM2090129"	"GSM2090130"	"GSM2090131"	"GSM2090132"	"GSM2090133"	"GSM2090134"	"GSM2090135"	"GSM2090136"	"GSM2090137"	"GSM2090138"	"GSM2090139"	"GSM2090140"	"GSM2090141"	"GSM2090142"	"GSM2090143"	"GSM2090144"	"GSM2090145"	"GSM2090146"	"GSM2090147"	"GSM2090148"	"GSM2090149"	"GSM2090150"	"GSM2090151"	"GSM2090152"	"GSM2090153"	"GSM2090154"	"GSM2090155"	"GSM2090156"	"GSM2090157"	"GSM2090158"	

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers from the previous output
# These identifiers start with "ILMN_" which indicates they are Illumina probe IDs
# rather than standard human gene symbols (like ACTB, TP53, etc.)
# Illumina IDs like ILMN_1343291 need to be mapped to human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe the gene identifiers in both dataframes
# The gene expression data has identifiers like 'ILMN_1343291' in its index
# The gene annotation data has a column 'ID' with similar values and a 'Symbol' column with gene symbols

# 2. Get a gene mapping dataframe by extracting the ID and Symbol columns
prob_col = 'ID'  # Column with Illumina probe IDs
gene_col = 'Symbol'  # Column with gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# Store the original probe count before mapping
original_probe_count = len(gene_data.index)

# 3. Apply the gene mapping to convert probe-level data to gene expression data
try:
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Print some info about the mapping result
    print(f"Original probe count: {original_probe_count}")
    print(f"Mapped gene count: {len(gene_data.index)}")
    print("First 10 genes after mapping:")
    print(gene_data.index[:10])
    
    # Save the processed gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")


Original probe count: 47295
Mapped gene count: 21459
First 10 genes after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE79263.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Define placeholder for convert_gender since it wasn't needed (gender_row is None)
convert_gender = None

# 2. Extract clinical features directly from the matrix file
try:
    # Get the file paths for the matrix file to extract clinical data
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get raw clinical data from the matrix file
    _, clinical_raw = get_background_and_clinical_data(matrix_file)
    
    # Verify clinical data structure
    print("Raw clinical data shape:", clinical_raw.shape)
    
    # Extract clinical features using the defined conversion functions
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_raw,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Clinical features:")
    print(clinical_features)
    
    # Save clinical features to file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, first 5 columns):")
    print(linked_data.iloc[:5, :5])
    
    # 4. Handle missing values
    print("Missing values before handling:")
    print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
    if 'Age' in linked_data.columns:
        print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
    if 'Gender' in linked_data.columns:
        print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
    
    gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
    print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
    print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
    
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # 5. Evaluate bias in trait and demographic features
    is_trait_biased = False
    if len(cleaned_data) > 0:
        trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        is_trait_biased = trait_biased
    else:
        print("No data remains after handling missing values.")
        is_trait_biased = True
    
    # 6. Final validation and save
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=cleaned_data,
        note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
    )
    
    # 7. Save if usable
    if is_usable and len(cleaned_data) > 0:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        cleaned_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Data was determined to be unusable or empty and was not saved")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # Handle the error case by still recording cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Mark as not available due to processing issues
        is_biased=True, 
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Error processing data: {str(e)}"
    )
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (20254, 94)
First few genes with their expression values after normalization:
          GSM2090086  GSM2090087  GSM2090088  GSM2090089  GSM2090090  \
Gene                                                                   
A1BG       37.186942   37.876860   36.393097   36.853984   41.675867   
A1BG-AS1   18.968444   27.694886   17.843437   19.722732   18.721735   
A1CF       52.382804   56.100401   54.195220   54.075285   56.799728   
A2M        58.343124   31.633324   20.444786   30.120143   31.559206   
A2ML1      19.408470   20.445197   17.789207   17.139441   17.707879   

          GSM2090091  GSM2090092  GSM2090093  GSM2090094  GSM2090095  ...  \
Gene                                                                  ...   
A1BG       36.573538   38.478149   36.868473   48.093884   45.978478  ...   
A1BG-AS1   19.211391   18.863080   20.727324   19.544603   17.718498  ...   
A1CF       59.610521   55.643605   54.333574   54.260276   53.901068  ...   
A2M 

Normalized gene data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/gene_data/GSE79263.csv


Raw clinical data shape: (5, 95)
Clinical features:
                             GSM2090086  GSM2090087  GSM2090088  GSM2090089  \
Duchenne_Muscular_Dystrophy         NaN         NaN         NaN         NaN   
Age                                80.0        78.0         NaN        79.0   

                             GSM2090090  GSM2090091  GSM2090092  GSM2090093  \
Duchenne_Muscular_Dystrophy         NaN         NaN         NaN         NaN   
Age                                19.0        17.0        15.0        73.0   

                             GSM2090094  GSM2090095  ...  GSM2090170  \
Duchenne_Muscular_Dystrophy         0.0         0.0  ...         1.0   
Age                                 NaN         NaN  ...         NaN   

                             GSM2090171  GSM2090172  GSM2090173  GSM2090174  \
Duchenne_Muscular_Dystrophy         1.0         1.0         1.0         1.0   
Age                                 NaN         NaN         NaN         NaN   

                 

Data shape after handling missing values: (86, 20256)
For the feature 'Duchenne_Muscular_Dystrophy', the least common label is '0.0' with 36 occurrences. This represents 41.86% of the dataset.
The distribution of the feature 'Duchenne_Muscular_Dystrophy' in this dataset is fine.

Quartiles for 'Age':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Duchenne_Muscular_Dystrophy/GSE79263.csv
